In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re

In [3]:
column_names = ['title','url','date']
new_df = pd.DataFrame(columns = column_names)

In [4]:
idx = 0

In [6]:
chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 122
Current browser version is 124.0.6367.209 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x010B8D03+51395]
	(No symbol) [0x01025F61]
	(No symbol) [0x00EDE13A]
	(No symbol) [0x00F0A021]
	(No symbol) [0x00F095DB]
	(No symbol) [0x00F054AE]
	(No symbol) [0x00F01E29]
	(No symbol) [0x00F3C655]
	(No symbol) [0x00F3C1FA]
	(No symbol) [0x00F33C26]
	(No symbol) [0x00F0C629]
	(No symbol) [0x00F0D40D]
	GetHandleVerifier [0x014368D3+3712147]
	GetHandleVerifier [0x01475CBA+3971194]
	GetHandleVerifier [0x01470FA8+3951464]
	GetHandleVerifier [0x01169D09+776393]
	(No symbol) [0x01031734]
	(No symbol) [0x0102C618]
	(No symbol) [0x0102C7C9]
	(No symbol) [0x0101DDF0]
	BaseThreadInitThunk [0x75F37BA9+25]
	RtlInitializeExceptionChain [0x776DBE3B+107]
	RtlClearBits [0x776DBDBF+191]


In [12]:
cur_month = 7
cur_year = 2017
target_year = 2023

while cur_year <= target_year:
    
    url = f"https://www.google.com/search?q=crude+oil&tbs=cdr:1,cd_min:{cur_month}/1/{cur_year},cd_max:{cur_month+5}/31/{cur_year}&tbm=nws"
    driver.get(url)
    
    time.sleep(30)
    
    while True:
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso > div >div>div>div')
        for news_div in news_results:
            try:
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                news_text_elements = news_div.find_elements(By.CSS_SELECTOR, 'a>div>div>div')
                news_item = [elem.text for elem in news_text_elements]
                if len(news_item) >= 5:
                    Link = news_link
                    Title = news_item[2]
                    Date = news_item[4]
                
                new_df.loc[idx] = [Title, Link, Date]
                idx += 1
            except Exception as e:
                print(e)
        
        try:
            next_button = driver.find_element(By.ID, 'pnnext')
            next_button.click()
            time.sleep(1)
        except NoSuchElementException:
            break
    if cur_month == 7:  
        cur_month = 1
        cur_year += 1
    else: 
        cur_month += 6


In [13]:
new_df

,title,url,date
0,"Oil in the Gulf, then and now",https://www.koreatimes.co.kr/www/nation/2024/0...,2010. 6. 1.
1,"Oil in the Gulf, then and now",https://www.koreatimes.co.kr/www/nation/2024/0...,2010. 6. 1.
2,Debt-laden Dubai discovers new offshore oil field,https://www.france24.com/en/20100204-debt-duba...,2010. 2. 4.
3,"Finance & Development, June 2010 - Reducing th...",https://www.imf.org/external/pubs/ft/fandd/201...,2010. 6. 16.
4,Record 2.5 bln barrels of oil futures traded T...,https://www.reuters.com/article/idUSLDE63D283/,2010. 4. 14.
...,...,...,...
6904,Crude oil price may hover at $80 per barrel ti...,https://www.deccanherald.com/business/crude-oi...,2023. 11. 30.
6905,How much oil is produced in the US?,https://usafacts.org/articles/how-much-oil-is-...,2023. 8. 1.
6906,More oil supertankers than ever are headed for...,https://finance.yahoo.com/news/more-oil-supert...,2023. 11. 6.
6907,Oil Prices on a March Toward $100 a Barrel,https://www.nytimes.com/2023/09/27/business/oi...,2023. 9. 27.


In [14]:
new_df.to_parquet('news_additional.parquet', engine = 'pyarrow', compression = 'gzip')